In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random  
from pandas.io.json import json_normalize
import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

from folium import plugins
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


CLIENT_ID = 'W4BTXLER5SBFSRSK1D1SNNEYL5VG0ZNUFSPHBUFNB130E4KB' 
CLIENT_SECRET = 'HPTR4RYBKVKCTKZ3ADQAY2QAG4WHDL2RM5DEKPOPD3ZE2RSG' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

print('CLIENT_SECRET:' + CLIENT_SECRET)

print("All Data has been imported.")


Your credentails:
CLIENT_ID: W4BTXLER5SBFSRSK1D1SNNEYL5VG0ZNUFSPHBUFNB130E4KB
CLIENT_SECRET:HPTR4RYBKVKCTKZ3ADQAY2QAG4WHDL2RM5DEKPOPD3ZE2RSG
All Data has been imported.


In [2]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [3]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Region', 
                  'Region Latitude', 
                  'Region Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [4]:
source_page = pd.read_csv('https://raw.githubusercontent.com/pawanahluwalia16/Applied-Data-Science-Capstone-Project/master/Canada%20Postal%20Code%20list.csv')

df = pd.DataFrame(source_page)
df = pd.DataFrame(source_page)
df.head()


,CA,T0A,Eastern Alberta (St. Paul),Alberta,AB,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,54.766,-111.7174,6
0,CA,T0B,Wainwright Region (Tofield),Alberta,AB,NaN,NaN,NaN,NaN,53.0727,-111.5816,6.0
1,CA,T0C,Central Alberta (Stettler),Alberta,AB,NaN,NaN,NaN,NaN,52.1431,-111.6941,5.0
2,CA,T0E,Western Alberta (Jasper),Alberta,AB,NaN,NaN,NaN,NaN,53.6758,-115.0948,5.0
3,CA,T0G,North Central Alberta (Slave Lake),Alberta,AB,NaN,NaN,NaN,NaN,55.6993,-114.4529,6.0
4,CA,T0H,Northwestern Alberta (High Level),Alberta,AB,NaN,NaN,NaN,NaN,57.5403,-116.9153,6.0


In [5]:
df.shape

(1655, 12)

In [6]:
del df['CA']
del df['AB']
del df['Unnamed: 5']
del df['Unnamed: 6']
del df['Unnamed: 7']
del df['Unnamed: 8']
del df['6']
del df['Alberta']

In [7]:
df.head()

,T0A,Eastern Alberta (St. Paul),54.766,-111.7174
0,T0B,Wainwright Region (Tofield),53.0727,-111.5816
1,T0C,Central Alberta (Stettler),52.1431,-111.6941
2,T0E,Western Alberta (Jasper),53.6758,-115.0948
3,T0G,North Central Alberta (Slave Lake),55.6993,-114.4529
4,T0H,Northwestern Alberta (High Level),57.5403,-116.9153


In [8]:
column_name = ['Postal Code','Region','Latitude','Longitude']
df.columns  = column_name
sp = pd.read_csv('https://cocl.us/Geospatial_data')
df.head()

,Postal Code,Region,Latitude,Longitude
0,T0B,Wainwright Region (Tofield),53.0727,-111.5816
1,T0C,Central Alberta (Stettler),52.1431,-111.6941
2,T0E,Western Alberta (Jasper),53.6758,-115.0948
3,T0G,North Central Alberta (Slave Lake),55.6993,-114.4529
4,T0H,Northwestern Alberta (High Level),57.5403,-116.9153


In [9]:
df_v=df[153:197]
df_v.reset_index(inplace=True)
del df_v['index']
df_v

,Postal Code,Region,Latitude,Longitude
0,V5K,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
1,V5L,Vancouver (North Grandview-Woodlands),49.2795,-123.0667
2,V5M,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398
3,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667
4,V5P,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683
5,V5R,Vancouver (South Renfrew-Collingwood),49.2397,-123.0407
6,V5S,Vancouver (Killarney),49.2175,-123.0380
7,V5T,Vancouver (East Mount Pleasant),49.2620,-123.0923
8,V5V,Vancouver (West Kensington / NE Riley Park-Lit...,49.2480,-123.0913
9,V5W,Vancouver (SE Riley Park-Little Mountain / SW ...,49.2327,-123.0917


In [10]:
m_p = folium.Map(location=[49.246292, -123.116226],zoom_start=12)

for l,L,Region in zip(df_v['Latitude'],df_v['Longitude'],df_v['Region']):
    label = '{}, {}'.format(Region, Region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [l,L],
    radius=2,
    color='red',
    fill=True,
    parse_html=False).add_to(m_p)
m_p

In [11]:
df_v.loc[0,'Region']

'Vancouver (North Hastings-Sunrise)'

In [12]:
neighborhood_latitude = df_v.loc[0, 'Latitude'] 
neighborhood_longitude = df_v.loc[0, 'Longitude'] 
neighborhood_name = df_v.loc[0, 'Region'] 

LIMIT = 100
radius = 500

url  = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(11)

/Users/pawanahluwalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Fair at the PNE,Fair,49.282971,-123.042109
1,Wooden Roller Coaster,Theme Park Ride / Attraction,49.281744,-123.035128
2,Playland,Theme Park,49.281924,-123.036258
3,Livestock Barns,Farm,49.284037,-123.039278
4,Empire Fields,Soccer Field,49.281870,-123.034135
5,Hastings Community Centre,Event Space,49.280778,-123.039176
6,PNE Amphitheatre,Amphitheater,49.283777,-123.037242
7,Plaza Beer Garden,Beer Garden,49.283701,-123.038785
8,Italia Bakery,Bakery,49.280940,-123.045885
9,Hastings Park,Park,49.281020,-123.042785


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


In [15]:
v_venues = getNearbyVenues(names=df_v['Region'],
                                   latitudes=df_v['Latitude'],
                                   longitudes=df_v['Longitude'])

Vancouver (North Hastings-Sunrise)
Vancouver (North Grandview-Woodlands)
Vancouver (South Hastings-Sunrise / North Renfrew-Collingwood)
Vancouver (South Grandview-Woodlands / NE Kensington)
Vancouver (SE Kensington / Victoria-Fraserview)
Vancouver (South Renfrew-Collingwood)
Vancouver (Killarney)
Vancouver (East Mount Pleasant)
Vancouver (West Kensington / NE Riley Park-Little Mountain)
Vancouver (SE Riley Park-Little Mountain / SW Kensington / NE Oakridge / North Sunset)
Vancouver (SE Oakridge / East Marpole / South Sunset)
Vancouver (West Mount Pleasant / West Riley Park-Little Mountain)
Vancouver (East Fairview / South Cambie)
Vancouver (Strathcona / Chinatown / Downtown Eastside)
Vancouver (NE Downtown / Harbour Centre / Gastown / Yaletown)
Vancouver (Waterfront / Coal Harbour / Canada Place)
Vancouver (South West End)
Vancouver (North West End / Stanley Park)
Vancouver (West Fairview / Granville Island / NE Shaughnessy)
Vancouver (NW Shaughnessy / East Kitsilano / Quilchena)
Vanco

In [20]:
v_venues.shape


(786, 7)

In [21]:
v_venues.groupby('Region').count()
v_venues.head()
v_venues.groupby('Region').count()
print('There are {} uniques categories.'.format(len(v_venues['Venue Category'].unique())))

There are 184 uniques categories.


In [22]:

v_onehot = pd.get_dummies(v_venues[['Venue Category']], prefix="", prefix_sep="")
v_onehot['Region'] = v_venues['Region'] 
fixed_columns = [v_onehot.columns[-1]] + list(v_onehot.columns[:-1])
v_onehot = v_onehot[fixed_columns]
v_onehot

,Region,Accessories Store,Airport Terminal,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Vancouver (North Hastings-Sunrise),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,Vancouver (Pacific Centre),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
782,Vancouver (Pacific Centre),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
783,Vancouver (Pacific Centre),0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
784,Vancouver (Pacific Centre),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
v_grouped = v_onehot.groupby('Region').mean().reset_index()
v_grouped.head()

,Region,Accessories Store,Airport Terminal,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Toy / Game Store,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,North Vancouver North Central,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,North Vancouver Northwest,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.263158,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,North Vancouver Northwest Central,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,North Vancouver South Central,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,North Vancouver Southwest,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.083333,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
num_top_venues = 5

for hood in v_grouped['Region']:
    print("----"+hood+"----")
    temp = v_grouped[v_grouped['Region'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----North Vancouver North Central----
             venue  freq
0   Scenic Lookout  0.33
1         Mountain  0.33
2            Trail  0.33
3           Office  0.00
4  Motorcycle Shop  0.00


----North Vancouver Northwest----
            venue  freq
0           Trail  0.26
1      Ski Chalet  0.11
2      Restaurant  0.11
3  Scenic Lookout  0.05
4        Mountain  0.05


----North Vancouver Northwest Central----
                           venue  freq
0                          Trail  0.67
1  Paper / Office Supplies Store  0.33
2              Accessories Store  0.00
3                   Noodle House  0.00
4                Motorcycle Shop  0.00


----North Vancouver South Central----
                 venue  freq
0              Butcher  0.25
1  Japanese Restaurant  0.25
2       Breakfast Spot  0.25
3       Sandwich Place  0.25
4               Office  0.00


----North Vancouver Southwest----
                 venue  freq
0          Coffee Shop  0.17
1       Discount Store  0.08
2     Toy / Game 

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Region'] = v_grouped['Region']

for ind in np.arange(v_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(v_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


# set number of clusters
kclusters = 5

v_grouped_clustering = v_grouped.drop('Region', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(v_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

v_merged = df_v

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
v_merged = v_merged.join(neighborhoods_venues_sorted.set_index('Region'), on='Region')

x = v_merged.dropna()
v_merged = x
v_merged.head(100)

,Postal Code,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V5K,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,1.0,Theme Park Ride / Attraction,Beer Garden,Theme Park,Event Space,Soccer Field,Inn,Sandwich Place,Bus Station,Portuguese Restaurant,Gas Station
1,V5L,Vancouver (North Grandview-Woodlands),49.2795,-123.0667,1.0,Café,Italian Restaurant,Asian Restaurant,Coffee Shop,Brewery,Sushi Restaurant,Theater,Chinese Restaurant,Deli / Bodega,Music Venue
2,V5M,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398,1.0,Bus Stop,Furniture / Home Store,Coffee Shop,Paper / Office Supplies Store,Event Service,Metro Station,Chinese Restaurant,Sandwich Place,Liquor Store,Storage Facility
3,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667,1.0,Café,Bakery,Beach,Bookstore,Farmers Market,Concert Hall,Lake,Church,Gym,Gym / Fitness Center
4,V5P,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683,1.0,Pizza Place,Fish Market,Gas Station,Pet Store,Pharmacy,Noodle House,Convenience Store,Restaurant,Motorcycle Shop,Middle Eastern Restaurant
5,V5R,Vancouver (South Renfrew-Collingwood),49.2397,-123.0407,1.0,Hotel,Park,Fish & Chips Shop,Bar,Asian Restaurant,Bus Stop,Event Service,Ethiopian Restaurant,Food Truck,Food Court
6,V5S,Vancouver (Killarney),49.2175,-123.0380,1.0,Bus Stop,Juice Bar,Salon / Barbershop,Farmers Market,Fast Food Restaurant,Liquor Store,Coffee Shop,Bank,Bakery,Grocery Store
7,V5T,Vancouver (East Mount Pleasant),49.2620,-123.0923,1.0,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Pizza Place,Sports Bar,Liquor Store,Café,Bar,Music Venue,Bakery
8,V5V,Vancouver (West Kensington / NE Riley Park-Lit...,49.2480,-123.0913,1.0,Coffee Shop,Pizza Place,Bakery,Café,Filipino Restaurant,Chinese Restaurant,Sandwich Place,Gourmet Shop,Greek Restaurant,Grocery Store
9,V5W,Vancouver (SE Riley Park-Little Mountain / SW ...,49.2327,-123.0917,1.0,Chinese Restaurant,Pizza Place,Tea Room,Dessert Shop,Coffee Shop,Fast Food Restaurant,Sandwich Place,Field,Bus Stop,Fried Chicken Joint


In [29]:
# create map
map_clusters = folium.Map(location=[49.246292, -123.116226], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(v_merged['Latitude'], v_merged['Longitude'], 
                                  v_merged['Region'],
                                  v_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
v_merged.loc[v_merged['Cluster Labels'] == 0, v_merged.columns[[1] + list(range(5, v_merged.shape[1]))]]

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Vancouver (West Kitsilano / Jericho),Park,Yoga Studio,Farm,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field
26,Vancouver (Chaldecutt / South University Endow...,Park,Yoga Studio,Farm,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field
34,North Vancouver Southwest Central,Park,Yoga Studio,Farm,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field


In [33]:
v_merged.loc[v_merged['Cluster Labels'] == 1, v_merged.columns[[1] + list(range(5, v_merged.shape[1]))]]

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Vancouver (North Hastings-Sunrise),Theme Park Ride / Attraction,Beer Garden,Theme Park,Event Space,Soccer Field,Inn,Sandwich Place,Bus Station,Portuguese Restaurant,Gas Station
1,Vancouver (North Grandview-Woodlands),Café,Italian Restaurant,Asian Restaurant,Coffee Shop,Brewery,Sushi Restaurant,Theater,Chinese Restaurant,Deli / Bodega,Music Venue
2,Vancouver (South Hastings-Sunrise / North Renf...,Bus Stop,Furniture / Home Store,Coffee Shop,Paper / Office Supplies Store,Event Service,Metro Station,Chinese Restaurant,Sandwich Place,Liquor Store,Storage Facility
3,Vancouver (South Grandview-Woodlands / NE Kens...,Café,Bakery,Beach,Bookstore,Farmers Market,Concert Hall,Lake,Church,Gym,Gym / Fitness Center
4,Vancouver (SE Kensington / Victoria-Fraserview),Pizza Place,Fish Market,Gas Station,Pet Store,Pharmacy,Noodle House,Convenience Store,Restaurant,Motorcycle Shop,Middle Eastern Restaurant
5,Vancouver (South Renfrew-Collingwood),Hotel,Park,Fish & Chips Shop,Bar,Asian Restaurant,Bus Stop,Event Service,Ethiopian Restaurant,Food Truck,Food Court
6,Vancouver (Killarney),Bus Stop,Juice Bar,Salon / Barbershop,Farmers Market,Fast Food Restaurant,Liquor Store,Coffee Shop,Bank,Bakery,Grocery Store
7,Vancouver (East Mount Pleasant),Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Pizza Place,Sports Bar,Liquor Store,Café,Bar,Music Venue,Bakery
8,Vancouver (West Kensington / NE Riley Park-Lit...,Coffee Shop,Pizza Place,Bakery,Café,Filipino Restaurant,Chinese Restaurant,Sandwich Place,Gourmet Shop,Greek Restaurant,Grocery Store
9,Vancouver (SE Riley Park-Little Mountain / SW ...,Chinese Restaurant,Pizza Place,Tea Room,Dessert Shop,Coffee Shop,Fast Food Restaurant,Sandwich Place,Field,Bus Stop,Fried Chicken Joint


In [34]:
v_merged.loc[v_merged['Cluster Labels'] == 2, v_merged.columns[[1] + list(range(5, v_merged.shape[1]))]]

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,North Vancouver Northwest Central,Trail,Paper / Office Supplies Store,Yoga Studio,Farm,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
41,West Vancouver West,Trail,Tapas Restaurant,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant


In [35]:
v_merged.loc[v_merged['Cluster Labels'] == 4, v_merged.columns[[1] + list(range(5,v_merged.shape[1]))]]

,Region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,West Vancouver Southeast,Bank,Pharmacy,Yoga Studio,Food Court,Food & Drink Shop,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field


In [36]:
pop = pd.read_csv('https://raw.githubusercontent.com/pawanahluwalia16/Applied-Data-Science-Capstone-Project/master/T1201EN.CSV')
dfp = pd.DataFrame(pop)
dfp = pd.DataFrame(pop)

del dfp['Incompletely enumerated Indian reserves and Indian settlements, 2016']
del dfp['Province or territory']
del dfp['Geographic name']

dfp.head()

,Geographic code,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,35151728.0,15412443.0,14072079.0
1,A0A,46587.0,26155.0,19426.0
2,A0B,19792.0,13658.0,8792.0
3,A0C,12587.0,8010.0,5606.0
4,A0E,22294.0,12293.0,9603.0


In [37]:
column_name = ['Postal Code','Population 2016',' Privte Dwellings 2016', 
              'Occupied Private Residents 2016']
dfp.columns  = column_name

dfp.head()

,Postal Code,Population 2016,Privte Dwellings 2016,Occupied Private Residents 2016
0,01,35151728.0,15412443.0,14072079.0
1,A0A,46587.0,26155.0,19426.0
2,A0B,19792.0,13658.0,8792.0
3,A0C,12587.0,8010.0,5606.0
4,A0E,22294.0,12293.0,9603.0


In [41]:
df_vp = pd.merge(v_merged,dfp,on = 'Postal Code', how = 'left')
df_vp

,Postal Code,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Population 2016,Privte Dwellings 2016,Occupied Private Residents 2016
0,V5K,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,1.0,Theme Park Ride / Attraction,Beer Garden,Theme Park,Event Space,Soccer Field,Inn,Sandwich Place,Bus Station,Portuguese Restaurant,Gas Station,23016.0,9129.0,8504.0
1,V5L,Vancouver (North Grandview-Woodlands),49.2795,-123.0667,1.0,Café,Italian Restaurant,Asian Restaurant,Coffee Shop,Brewery,Sushi Restaurant,Theater,Chinese Restaurant,Deli / Bodega,Music Venue,19720.0,10810.0,10272.0
2,V5M,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398,1.0,Bus Stop,Furniture / Home Store,Coffee Shop,Paper / Office Supplies Store,Event Service,Metro Station,Chinese Restaurant,Sandwich Place,Liquor Store,Storage Facility,23053.0,8909.0,8241.0
3,V5N,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667,1.0,Café,Bakery,Beach,Bookstore,Farmers Market,Concert Hall,Lake,Church,Gym,Gym / Fitness Center,35045.0,15331.0,14351.0
4,V5P,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683,1.0,Pizza Place,Fish Market,Gas Station,Pet Store,Pharmacy,Noodle House,Convenience Store,Restaurant,Motorcycle Shop,Middle Eastern Restaurant,31920.0,11594.0,10466.0
5,V5R,Vancouver (South Renfrew-Collingwood),49.2397,-123.0407,1.0,Hotel,Park,Fish & Chips Shop,Bar,Asian Restaurant,Bus Stop,Event Service,Ethiopian Restaurant,Food Truck,Food Court,46565.0,19034.0,16927.0
6,V5S,Vancouver (Killarney),49.2175,-123.0380,1.0,Bus Stop,Juice Bar,Salon / Barbershop,Farmers Market,Fast Food Restaurant,Liquor Store,Coffee Shop,Bank,Bakery,Grocery Store,28773.0,11155.0,10450.0
7,V5T,Vancouver (East Mount Pleasant),49.2620,-123.0923,1.0,Sushi Restaurant,Vietnamese Restaurant,Ethiopian Restaurant,Pizza Place,Sports Bar,Liquor Store,Café,Bar,Music Venue,Bakery,23911.0,13710.0,12924.0
8,V5V,Vancouver (West Kensington / NE Riley Park-Lit...,49.2480,-123.0913,1.0,Coffee Shop,Pizza Place,Bakery,Café,Filipino Restaurant,Chinese Restaurant,Sandwich Place,Gourmet Shop,Greek Restaurant,Grocery Store,24560.0,10065.0,9253.0
9,V5W,Vancouver (SE Riley Park-Little Mountain / SW ...,49.2327,-123.0917,1.0,Chinese Restaurant,Pizza Place,Tea Room,Dessert Shop,Coffee Shop,Fast Food Restaurant,Sandwich Place,Field,Bus Stop,Fried Chicken Joint,19706.0,7935.0,7111.0


In [39]:

lat = 49.246292
lon =  -123.116226
m_pv = folium.Map(location=[lat,lon],zoom_start=2)

markers_colors = []
for index,lat,lon, Region, cluster, pop in zip(range(len(df_vp),df_vp['Latitude'],df_vp['Longitude'],df_vp['Region'],df_vp['Cluster Labels'],df_vp['Population, 2016']):
    
    label = folium.Popup(str(Region) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= df_vp.iloc[index]['Population 2016']
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

SyntaxError: invalid syntax (<ipython-input-39-a47ef9df1dac>, line 6)

In [40]:
# create map
map_clusters = folium.Map(location=[49.246292, -123.116226], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for index, lat,lon,Region,cluster, pop in zip(range(len(df_vp)),df_vp['Latitude'], df_vp['Longitude'], 
                                  df_vp['Region'],
                                  df_vp['Cluster Labels'], df_vp['Population 2016']):
    label = folium.Popup(str(Region) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= df_vp.iloc[index]['Population 2016']*0.001,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters       